In [0]:
import nltk
import time
import heapq
import pandas as pd
import numpy as np
import bisect
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer

nltk.download('rslp')
nltk.download('punkt')
nltk.download('stopwords')

_stopwords = set(stopwords.words('portuguese'))

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Carregando Índice Invertido

In [0]:
data = pd.read_csv("https://raw.githubusercontent.com/velmer/ri_lab_01/master/output/results.csv")
data.head()

,title,sub_title,author,date,section,text,url
0,VÍDEO – “Estarei nas ruas com a militância do ...,Não possui,Pedro Zambarda de Araujo,05/04/2019 22:44:19,Não possui,Não ficaremos calados diante da injustiça. Lul...,https://www.diariodocentrodomundo.com.br/video...
1,Essencial do DCM – Bolsonaro diz que não nasce...,Não possui,Pedro Zambarda de Araujo,05/04/2019 21:00:02,Não possui,O jornalismo do DCM precisa de você para conti...,https://www.diariodocentrodomundo.com.br/essen...
2,Conheça o deputado árabe que pode derrubar o p...,Não possui,Diario do Centro do Mundo,05/04/2019 22:00:56,Não possui,A revista francesa L’Obs desta semana traz um ...,https://www.diariodocentrodomundo.com.br/conhe...
3,"VÍDEO – Guedes, as tchutchucas e a lacrada des...",Não possui,Nathalí Macedo,05/04/2019 16:23:14,Não possui,O jornalismo do DCM precisa de você para conti...,https://www.diariodocentrodomundo.com.br/video...
4,O que está por trás da articulação para Lula t...,Não possui,Joaquim de Carvalho,05/04/2019 21:42:17,Não possui,Veja o vídeo “Foi para isso que prenderam o Lu...,https://www.diariodocentrodomundo.com.br/o-que...


# Construindo Índice Invertido

In [0]:
tknz = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê\-]{3,27})')
tokenData = pd.DataFrame(data['text'], columns=['text'])
tokenData['tokenized_text'] = data.apply(lambda row: tknz.tokenize(row['text'].lower()), axis=1)

In [0]:
def is_tokenized_word(word):
  """
    Returns if the specified word matches all requirements
    to be a valid token.
  """
  return len(word) > 1 and any(c.isalnum() for c in word)

def build_index():
  index = {}
  num_document = 0

  for text in tokenData.tokenized_text:
    num_document += 1
    words_set = set(text)
    for word in words_set:
      if is_tokenized_word(word) and word.lower() not in _stopwords:
        if word not in index.keys():
          index[word] = []
        index[word].append((num_document, text.count(word)))

  return index

In [0]:
indexed_data = pd.DataFrame()
index = build_index()
indexed_data['Word'] = [word for word in index.keys()]
indexed_data['Documents (With Count)'] = [docs for docs in index.values()]
indexed_data.to_csv('indexed_data.csv', encoding='utf-8', index=False)
indexed_data.head(10)

,Word,Documents (With Count)
0,vitória,"[(1, 1), (3, 1), (57, 1), (97, 1), (138, 2), (..."
1,diante,"[(1, 1), (17, 1), (27, 1), (52, 2), (67, 1), (..."
2,psol,"[(1, 1), (138, 2), (294, 2), (320, 1), (519, 1..."
3,julianopsol,"[(1, 1)]"
4,direto,"[(1, 1), (55, 1), (118, 1), (130, 1), (147, 1)..."
5,injustiça,"[(1, 1), (76, 1), (102, 1), (146, 1), (208, 2)..."
6,dessa,"[(1, 1), (27, 2), (55, 2), (56, 1), (58, 1), (..."
7,bolsonaro,"[(1, 1), (6, 1), (10, 16), (11, 1), (13, 3), (..."
8,ficaremos,"[(1, 1)]"
9,nacional,"[(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1..."


# Questão 1

In [0]:
queries = ["presidente", "bolsonaro", "país", "futebol", "site"]

In [0]:
def intersection(list1, list2):
  set1 = set(list1)
  set2 = set(list2)
  return set1 & set2

def all_occurrances_word(word, index2):
  return index2[word]

def num_occurrances_word(word, index):
  return len(all_occurrances_word(word, index))

def all_occurrances_bigram(word1, word2, index):
  return intersection(all_occurrances_word(word1, index), all_occurrances_word(word2, index))

def num_occurrances_bigram(word1, word2, index):
  return len(all_occurrances_bigram(word1, word2, index))

In [0]:
def calc_mim(a, b, index):
  n_a = num_occurrances_word(a, index)
  n_b = num_occurrances_word(b, index)
  if (n_a == 0 or n_b == 0):
    return (0, b)
  n_ab = num_occurrances_bigram(a, b, index)
  mim = n_ab / (n_a * n_b)  
  return (mim,b)
  
def calc_emim(a, b, n, index):
  n_a = num_occurrances_word(a, index)
  n_b = num_occurrances_word(b, index)
  n_ab = num_occurrances_bigram(a, b, index)
  quo = (n_a * n_b)
  exp = n * (n_ab / quo) if quo != 0 else 0
  emim = n_ab * np.log10(exp) if exp != 0 else 0
  return (emim, b)
  
def calc_x2(a, b, n, index):
  n_a = num_occurrances_word(a, index)
  n_b = num_occurrances_word(b, index)
  if (n_a == 0 or n_b == 0):
    return (0, b)
  n_ab = num_occurrances_bigram(a, b, index)
  x2 = (n_ab - (1 / n) * n_a * n_b) ** 2 / (n_a * n_b)
  return (x2, b)
  
def calc_dice(a, b, index):
  n_a = num_occurrances_word(a, index)
  n_b = num_occurrances_word(b, index)
  if (n_a == 0 and n_b == 0):
    return (0, b)
  n_ab = num_occurrances_bigram(a, b, index)
  dice = n_ab / (n_a + n_b)
  return (dice, b)

In [0]:
def create_most_related_metrics_table(query, index, n):
  top_10_table = pd.DataFrame()
  top_10_MIM = []
  top_10_EMIM = []
  top_10_X2 = []
  top_10_Dice = []
  
  for word in index:
    if word != query:
      bisect.insort(top_10_MIM, calc_mim(query, word, index))
      bisect.insort(top_10_EMIM,calc_emim(query, word, n, index))
      bisect.insort(top_10_X2,calc_x2(query, word, n, index))
      bisect.insort(top_10_Dice,calc_dice(query, word, index))
  
  top_10_MIM.reverse()
  top_10_EMIM.reverse()
  top_10_X2.reverse()
  top_10_Dice.reverse()
  
  top_10_table['MIM'] = [word[1] for word in top_10_MIM]
  top_10_table['EMIM'] = [word[1] for word in top_10_EMIM]
  top_10_table['X²'] = [word[1] for word in top_10_X2]
  top_10_table['Dice'] = [word[1] for word in top_10_Dice]
  
  return top_10_table.head(10)

In [0]:
n = data.text.count()
table_query0 = create_most_related_metrics_table(queries[0], index, n)
table_query1 = create_most_related_metrics_table(queries[1], index, n)
table_query2 = create_most_related_metrics_table(queries[2], index, n)
table_query3 = create_most_related_metrics_table(queries[3], index, n)
table_query4 = create_most_related_metrics_table(queries[4], index, n)

### Top-10 palavras associadas à query "presidente"

In [0]:
table_query0

,MIM,EMIM,X²,Dice
0,óbvios,provocar,vida,jair
1,ínfimo,criticou,nacional,ser
2,ínfima,autoritário,precisa,bolsonaro
3,é-lhes,abandonar,dcm,hoje
4,ânsia,francês,ponto,tudo
5,álcool,começo,continuar,marcando
6,zoológico,articulador,marcando,jornalismo
7,yousafzai,evidentemente,jornalismo,ponto
8,yasser,trato,sobre,continuar
9,yann,itaipu,globo,sido


### Top-10 palavras associadas à query "bolsonaro"

In [0]:
table_query1

,MIM,EMIM,X²,Dice
0,ímpeto,injustiça,nacional,jair
1,ícones,desmonte,vida,presidente
2,ânsia,mando,ponto,jornalismo
3,ángel,jair,continuar,marcando
4,álcool,protestos,precisa,dcm
5,zusammen,manifestantes,marcando,precisa
6,zelar,instituições,dcm,continuar
7,yorko,risos,jornalismo,ponto
8,xerife,pensava,ainda,vida
9,wolf,constitui,sobre,ser


### Top-10 palavras associadas à query "país"

In [0]:
table_query2

,MIM,EMIM,X²,Dice
0,únicas,dados,nacional,ainda
1,ótica,fechar,vida,todo
2,óbvios,considera,dcm,grande
3,ínfimo,cerca,precisa,todos
4,índias,devem,ponto,dia
5,ímpeto,bolsonaros,marcando,pode
6,éticas,querendo,continuar,onde
7,zezé,concorrer,jornalismo,hoje
8,zeidler,conseguiu,fechar,segundo
9,zavascki,maiores,regiões,tudo


### Top-10 palavras associadas à query "futebol"

In [0]:
table_query3

,MIM,EMIM,X²,Dice
0,ángel,saídas,saídas,saídas
1,zusammen,coronel,ángel,coronel
2,yorko,categorias,zusammen,categorias
3,xingando,atender,yorko,atender
4,violados,italiano,xingando,italiano
5,vigilkapelle,flamengo,violados,flamengo
6,vigilia,salvador,vigilkapelle,ángel
7,victoria,prêmio,vigilia,zusammen
8,viadutos,núcleo,victoria,yorko
9,verificar,ocorrido,viadutos,xingando


### Top-10 palavras associadas à query "site"

In [0]:
table_query4

,MIM,EMIM,X²,Dice
0,ótica,doação,doação,doação
1,íntima,seguirmos,seguirmos,seguirmos
2,zimmerman,combatendo,combatendo,combatendo
3,wi-fi,loira,loira,fake
4,wedding,fake,moto,loira
5,warren,diferença,sérias,diferença
6,vê-las,sair,punida,sair
7,volatilidade,independência,intervalos,independência
8,visível,moto,indagado,obrigado
9,visualizações,obrigado,falsos,faça


Podemos observar com certa facilidade que o MIM não obtém bons resultados, retornando comumente palavras de outros idiomas e outras que são difíceis de se imaginar um contexto no qual estariam relacionadas a query, como é o caso da palavra "zoológico" para a query "presidente".

As outras três métricas estão com resultados mais similares, como é o caso da query "site", que EMIM, X² e Dice retornam as mesmas palavras na mesma sequência: "doação", "seguirmos" e "combatendo".

Porém, podemos observar um melhor desempenho por parte das métricas EMIM e Dice, pois X² não obteve bons resultados no geral, como por exemplo na query "futebol", que a métrica retornou palavras em outros idiomas e outras que não estão relacionadas à query como "viadutos", enquanto EMIM e Dice retornaram 6 palavras em comum, inclusive na mesma ordem.

Para decidir a melhor entre as duas, considerei a simplicidade da métrica, sendo Dice consideravelmentem ais simples que EMIM (alcançando resultados bastante similares) e a relação entre as duas primeiras queries "presidente" e "bolsonaro", nas quais a métrica Dice encontrou "bolsonaro" como resultado da primeira e encontrou "presidente" como resultado da segunda query.

Nesse sentido, julgo que Dice foi a métrica que obteve melhores resultados.

# Questão 2

In [0]:
def term_at_time(query, index, k):
    a = {}
    document_count_list = []
    r = []
    for word in query.split():
        document_count_list.append(index[word])
    for document_count in document_count_list:
        for post in document_count:
            d = post[0]
            freq = post[1]
            if (d in a.keys()):
                a[d] += freq
            else:
                a[d] = freq
    for (d, ad) in a.items():
        sd = ad
        heapq.heappush(r, (sd, d))

    return heapq.nlargest(k, r)

In [0]:
def index_with_ntop_docs(query, k):
  ntop_docs_ranking = {} 
  ntop_docs_indexes = [doc for score, doc in term_at_time(query, index, k)]
  
  for word in index:
    ntop_docs_ranking[word] = []
    for doc_score in index[word]:
      if doc_score[0] in ntop_docs_indexes:
        ntop_docs_ranking[word].append(doc_score)
        
  return ntop_docs_ranking

In [0]:
k = 3

index_with_top_3_docs_query0 = index_with_ntop_docs(queries[0], k)
index_with_top_3_docs_query1 = index_with_ntop_docs(queries[1], k)
index_with_top_3_docs_query2 = index_with_ntop_docs(queries[2], k)
index_with_top_3_docs_query3 = index_with_ntop_docs(queries[3], k)
index_with_top_3_docs_query4 = index_with_ntop_docs(queries[4], k)

table_top3_query0 = create_most_related_metrics_table(queries[0], index_with_top_3_docs_query0, k)
table_top3_query1 = create_most_related_metrics_table(queries[1], index_with_top_3_docs_query1, k)
table_top3_query2 = create_most_related_metrics_table(queries[2], index_with_top_3_docs_query2, k)
table_top3_query3 = create_most_related_metrics_table(queries[3], index_with_top_3_docs_query3, k)
table_top3_query4 = create_most_related_metrics_table(queries[4], index_with_top_3_docs_query4, k)

### Comparação com Top-3 documentos para query "presidente"

In [0]:
comparation_table_top3_query0 = pd.DataFrame()
comparation_table_top3_query0["Dice (All Documents)"] = table_query0["Dice"]
comparation_table_top3_query0["Dice (Top-3 Documents)"] = table_top3_query0["Dice"]
comparation_table_top3_query0

,Dice (All Documents),Dice (Top-3 Documents)
0,jair,cni
1,ser,país
2,bolsonaro,útil
3,hoje,úteis
4,tudo,únicos
5,marcando,único
6,jornalismo,únicas
7,ponto,única
8,continuar,últimos
9,sido,último


### Comparação com Top-3 documentos para query "bolsonaro"

In [0]:
comparation_table_top3_query1 = pd.DataFrame()
comparation_table_top3_query1["Dice (All Documents)"] = table_query1["Dice"]
comparation_table_top3_query1["Dice (Top-3 Documents)"] = table_top3_query1["Dice"]
comparation_table_top3_query1

,Dice (All Documents),Dice (Top-3 Documents)
0,jair,projeto
1,presidente,porque
2,jornalismo,útil
3,marcando,úteis
4,dcm,únicos
5,precisa,único
6,continuar,únicas
7,ponto,única
8,vida,últimos
9,ser,último


### Comparação com Top-3 documentos para query "país"

In [0]:
comparation_table_top3_query2 = pd.DataFrame()
comparation_table_top3_query2["Dice (All Documents)"] = table_query2["Dice"]
comparation_table_top3_query2["Dice (Top-3 Documents)"] = table_top3_query2["Dice"]
comparation_table_top3_query2

,Dice (All Documents),Dice (Top-3 Documents)
0,ainda,mentira
1,todo,cni
2,grande,sentido
3,todos,presidente
4,dia,governo
5,pode,útil
6,onde,úteis
7,hoje,únicos
8,segundo,único
9,tudo,únicas


### Comparação com Top-3 documentos para query "futebol"

In [0]:
comparation_table_top3_query3 = pd.DataFrame()
comparation_table_top3_query3["Dice (All Documents)"] = table_query3["Dice"]
comparation_table_top3_query3["Dice (Top-3 Documents)"] = table_top3_query3["Dice"]
comparation_table_top3_query3

,Dice (All Documents),Dice (Top-3 Documents)
0,saídas,zona
1,coronel,vestir
2,categorias,total
3,atender,seleção
4,italiano,repressão
5,flamengo,muitas
6,ángel,mostram
7,zusammen,mico
8,yorko,manto
9,xingando,joão


### Comparação com Top-3 documentos para query "site"

In [0]:
comparation_table_top3_query4 = pd.DataFrame()
comparation_table_top3_query4["Dice (All Documents)"] = table_query4["Dice"]
comparation_table_top3_query4["Dice (Top-3 Documents)"] = table_top3_query4["Dice"]
comparation_table_top3_query4

,Dice (All Documents),Dice (Top-3 Documents)
0,doação,outras
1,seguirmos,fake
2,combatendo,além
3,fake,vinham
4,loira,venceu
5,diferença,usuários
6,sair,tratasse
7,independência,tratado
8,obrigado,tempo
9,faça,telefone


In [0]:
k = 5

index_with_top_5_docs_query0 = index_with_ntop_docs(queries[0], k)
index_with_top_5_docs_query1 = index_with_ntop_docs(queries[1], k)
index_with_top_5_docs_query2 = index_with_ntop_docs(queries[2], k)
index_with_top_5_docs_query3 = index_with_ntop_docs(queries[3], k)
index_with_top_5_docs_query4 = index_with_ntop_docs(queries[4], k)

table_top5_query0 = create_most_related_metrics_table(queries[0], index_with_top_5_docs_query0, k)
table_top5_query1 = create_most_related_metrics_table(queries[1], index_with_top_5_docs_query1, k)
table_top5_query2 = create_most_related_metrics_table(queries[2], index_with_top_5_docs_query2, k)
table_top5_query3 = create_most_related_metrics_table(queries[3], index_with_top_5_docs_query3, k)
table_top5_query4 = create_most_related_metrics_table(queries[4], index_with_top_5_docs_query4, k)

### Comparação com Top-5 documentos para query "presidente"

In [0]:
comparation_table_top5_query0 = pd.DataFrame()
comparation_table_top5_query0["Dice (All Documents)"] = table_query0["Dice"]
comparation_table_top5_query0["Dice (Top-5 Documents)"] = table_top5_query0["Dice"]
comparation_table_top5_query0

,Dice (All Documents),Dice (Top-5 Documents)
0,jair,jair
1,ser,ser
2,bolsonaro,bolsonaro
3,hoje,hoje
4,tudo,tudo
5,marcando,marcando
6,jornalismo,jornalismo
7,ponto,ponto
8,continuar,continuar
9,sido,sido


### Comparação com Top-5 documentos para query "bolsonaro"

In [0]:
comparation_table_top5_query1 = pd.DataFrame()
comparation_table_top5_query1["Dice (All Documents)"] = table_query1["Dice"]
comparation_table_top5_query1["Dice (Top-5 Documents)"] = table_top5_query1["Dice"]
comparation_table_top5_query1

,Dice (All Documents),Dice (Top-5 Documents)
0,jair,jair
1,presidente,presidente
2,jornalismo,jornalismo
3,marcando,marcando
4,dcm,dcm
5,precisa,precisa
6,continuar,continuar
7,ponto,ponto
8,vida,vida
9,ser,ser


### Comparação com Top-5 documentos para query "país"

In [0]:
comparation_table_top5_query2 = pd.DataFrame()
comparation_table_top5_query2["Dice (All Documents)"] = table_query2["Dice"]
comparation_table_top5_query2["Dice (Top-5 Documents)"] = table_top5_query2["Dice"]
comparation_table_top5_query2

,Dice (All Documents),Dice (Top-5 Documents)
0,ainda,ainda
1,todo,todo
2,grande,grande
3,todos,todos
4,dia,dia
5,pode,pode
6,onde,onde
7,hoje,hoje
8,segundo,segundo
9,tudo,tudo


### Comparação com Top-5 documentos para query "futebol"

In [0]:
comparation_table_top5_query3 = pd.DataFrame()
comparation_table_top5_query3["Dice (All Documents)"] = table_query3["Dice"]
comparation_table_top5_query3["Dice (Top-5 Documents)"] = table_top5_query3["Dice"]
comparation_table_top5_query3

,Dice (All Documents),Dice (Top-5 Documents)
0,saídas,saídas
1,coronel,coronel
2,categorias,categorias
3,atender,atender
4,italiano,italiano
5,flamengo,flamengo
6,ángel,ángel
7,zusammen,zusammen
8,yorko,yorko
9,xingando,xingando


### Comparação com Top-5 documentos para query "site"

In [0]:
comparation_table_top5_query4 = pd.DataFrame()
comparation_table_top5_query4["Dice (All Documents)"] = table_query4["Dice"]
comparation_table_top5_query4["Dice (Top-5 Documents)"] = table_top5_query4["Dice"]
comparation_table_top5_query4

,Dice (All Documents),Dice (Top-5 Documents)
0,doação,doação
1,seguirmos,seguirmos
2,combatendo,combatendo
3,fake,fake
4,loira,loira
5,diferença,diferença
6,sair,sair
7,independência,independência
8,obrigado,obrigado
9,faça,faça


In [0]:
k = 10

index_with_top_10_docs_query0 = index_with_ntop_docs(queries[0], k)
index_with_top_10_docs_query1 = index_with_ntop_docs(queries[1], k)
index_with_top_10_docs_query2 = index_with_ntop_docs(queries[2], k)
index_with_top_10_docs_query3 = index_with_ntop_docs(queries[3], k)
index_with_top_10_docs_query4 = index_with_ntop_docs(queries[4], k)

table_top10_query0 = create_most_related_metrics_table(queries[0], index_with_top_10_docs_query0, k)
table_top10_query1 = create_most_related_metrics_table(queries[1], index_with_top_10_docs_query1, k)
table_top10_query2 = create_most_related_metrics_table(queries[2], index_with_top_10_docs_query2, k)
table_top10_query3 = create_most_related_metrics_table(queries[3], index_with_top_10_docs_query3, k)
table_top10_query4 = create_most_related_metrics_table(queries[4], index_with_top_10_docs_query4, k)

### Comparação com Top-10 documentos para query "presidente"

In [0]:
comparation_table_top10_query0 = pd.DataFrame()
comparation_table_top10_query0["Dice (All Documents)"] = table_query0["Dice"]
comparation_table_top10_query0["Dice (Top-10 Documents)"] = table_top10_query0["Dice"]
comparation_table_top10_query0

,Dice (All Documents),Dice (Top-10 Documents)
0,jair,cni
1,ser,força
2,bolsonaro,estudantes
3,hoje,grandes
4,tudo,dois
5,marcando,sobre
6,jornalismo,grande
7,ponto,país
8,continuar,bolsonaro
9,sido,útil


### Comparação com Top-10 documentos para query "bolsonaro"

In [0]:
comparation_table_top10_query1 = pd.DataFrame()
comparation_table_top10_query1["Dice (All Documents)"] = table_query1["Dice"]
comparation_table_top10_query1["Dice (Top-10 Documents)"] = table_top10_query1["Dice"]
comparation_table_top10_query1

,Dice (All Documents),Dice (Top-10 Documents)
0,jair,projeto
1,presidente,porque
2,jornalismo,útil
3,marcando,úteis
4,dcm,únicos
5,precisa,único
6,continuar,únicas
7,ponto,única
8,vida,últimos
9,ser,último


### Comparação com Top-10 documentos para query "país"

In [0]:
comparation_table_top10_query2 = pd.DataFrame()
comparation_table_top10_query2["Dice (All Documents)"] = table_query2["Dice"]
comparation_table_top10_query2["Dice (Top-10 Documents)"] = table_top10_query2["Dice"]
comparation_table_top10_query2

,Dice (All Documents),Dice (Top-10 Documents)
0,ainda,presidente
1,todo,governo
2,grande,mentira
3,todos,medo
4,dia,cni
5,pode,acadêmicos
6,onde,nicolás
7,hoje,aprovação
8,segundo,serem
9,tudo,sentido


### Comparação com Top-10 documentos para query "futebol"

In [0]:
comparation_table_top10_query3 = pd.DataFrame()
comparation_table_top10_query3["Dice (All Documents)"] = table_query3["Dice"]
comparation_table_top10_query3["Dice (Top-10 Documents)"] = table_top10_query3["Dice"]
comparation_table_top10_query3

,Dice (All Documents),Dice (Top-10 Documents)
0,saídas,jair
1,coronel,outros
2,categorias,sendo
3,atender,saídas
4,italiano,salvador
5,flamengo,saber
6,ángel,respeito
7,zusammen,quais
8,yorko,prêmio
9,xingando,núcleo


### Comparação com Top-10 documentos para query "site"

In [0]:
comparation_table_top10_query4 = pd.DataFrame()
comparation_table_top10_query4["Dice (All Documents)"] = table_query4["Dice"]
comparation_table_top10_query4["Dice (Top-10 Documents)"] = table_top10_query4["Dice"]
comparation_table_top10_query4

,Dice (All Documents),Dice (Top-10 Documents)
0,doação,vida
1,seguirmos,outras
2,combatendo,primeiro
3,fake,fake
4,loira,coisa
5,diferença,sei
6,sair,psl
7,independência,policial
8,obrigado,moto
9,faça,coleta


Para as consultas expandidas apenas com os top-3 documentos considero que a qualidade diminui, visto que temos muitas palavras similares entre si, mas que não possuem relação com a query em questão.

Já para os top-5 documentos os resultados são excelentes, pois eles são exatamente os mesmos dos resultados obtidos com todos os documentos da coleção. Resultado bastante inesperado, visto que o desempenho com os top-3 elementos deixou a desejar.

Para os top-10 documentos, ao contrário do senso comum ao observar o top-5 (imaginar que melhoraria ou se manteria igual), a qualidade dos resultados diminuiu. Para a query "bolsonaro", o desempenho da métrica Dice foi bastante similar aos resultados obtidos com os top-3 documentos, os quais não foram bons.